<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/Variational_Auto_encoder_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import (
    layers,
    models,
    datasets,
    callbacks,
    losses,
    optimizers,
    metrics,
)

from scipy.stats import norm

#0. Parameters

In [ ]:
IMAGE_SIZE = 32
BATCH_SIZE = 100
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 2
EPOCHS = 5
BETA = 500

#Prepare the data

In [ ]:
#Load the data
(x_train , y_train), (x_test , y_test) = datasets.fashion_mnist.load_data()

In [ ]:
#Preprocess the data
def preprocess(imgs):
  """
  normalize and reshape the image
  """
  imgs = imgs.astype("float32")
  imgs = np.pad(imgs, ((0,0) , (2,2), (2,2)), constant_values = 0.0)
  imgs = np.expand_dims(imgs, -1)
  return imgs

x_train = preprocess(x_train)
x_test = preprocess(x_test)

In [ ]:
#Show some items of clothings from the training set
x_train

#2. Build the variational autoencoder

In [ ]:
class Sampling (layers.Layer):
  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = K.random_normal(shape = (batch, dim))
    return z_mean +tf.exp(0.5 * z_log_var) * epsilon


In [ ]:
#Encoder
encoder_input = layers.Input(
    shape = (IMAGE_SIZE , IMAGE_SIZE, 1), name = "encoder_input"
)
x = layers.Conv2D(32,(3,3), strides = 2, activation = "relu" , padding = "same")(
    encoder_input
)
x = layers.Conv2D(64, (3,3), srides = 2, activation = "relu", padding = "same")(x)
x = layers.Conv2D(128, (3,3), strides = 2,activation = "relu" , padding = "same")(x)
shape_before_flattening = K.int_shape(x)[1:]

x = layers.Flatten()(x)
z_mean = layers.Dense(EMBEDDING_DIM , name = "z_mean")(x)
z_log_var = layers.Dense(EMBEDDING_DIM, name = "z_log_var")(x)
z = Sampling()([z_mean , z_log_var])

encoder = models.Model(encoder_input, [z_mean , z_log_var, z] , name = "encoder")
encoder.summary()

In [ ]:
#Decoder
decoder_input = layers.Input(shape = (EMBEDDING_DIM,) , name = "decoder_input")
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(
    128, (3,3), strides = 2, activation = "relu" , padding = "same"
)(x)
x = layers.Conv2DTranspose(
    64, (3,3) , strides= 2, activation = "relu" , padding = "same"
)(x)
x = layers.Conv2DTranspose(
    32, (3,3) , strides = 2, activation = "relu" , padding = "same"
)(x)
decoder_output = layers.Conv2D(
    1,
    strides = 1,
    activation = "sigmoid",
    padding = "same",
    name = "decoder_output",
)(x)

decoder = models.Model(decoder_input, decoder_output)
decoder.summary()

In [ ]:
class VAE(models.Model):
  def __init__(self,encoder,decoder , **kwargs):
    super(VAE, self).__init__(**kwargs)
    self.encoder = encoder
    self.decoder = decoder
    self.total_loss_tracker = metrics.Mean(name = "total_loss")
    self.reconstruction_loss_tracker = metrics.Mean(
        name = "reconstruction_loss"
    )
    self.kl_loss_tracker = metrics.Mean(name = "kl_loss")

  @property
  def metrics(self):
    return[
        self.total_loss_tracker,
        self.reconstruction_loss_tracker,
        self.kl_loss_tracker,
    ]

  def call(self, inputs):
    """ Call the model on a particular input"""
    z_mean , z_log_var , z = encoder(inputs)
    reconstruction = decoder(z)
    return z_mean , z_log_var, reconstruction

  def train_step(self, data):
    """Step run during training """
    with tf.GradientTape() as tape:
      z_mean , z_log_var, reconstruction = self(data)
      reconstruction_loss = tf.reduce_mean(
          BETA
          * losses.binary_crossentropy(
              data, reconstruction, axis = (1,2,3)
          )
      )
      kl_loss = tf.reduce_mean(
          tf.reduce_sum(
              -0.5
              * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
              axis = 1,
          )
      )
      total_loss = reconstruction_loss + kl_loss

    grads = tape.gradient(total_loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

    self.total_loss_tracker.update_state(total_loss)
    self.reconstruction_loss_tracker.update_state(reconstruction_loss)
    self.kl_loss_tracker.update_state(kl_loss)

    return {m.name : m.result() for m in self.metrics}

  def test_step(self, data):
    """Step run during validation"""
    if isinstance(data, tuple):
      data = data[0]

    z_mean , z_log_var , reconstruction = self(data)
    reconstruction_loss = tf.reduce_mean(
        BETA
        * losses.binary_crossentropy(data, reconstruction, axis = (1,2,3))
    )
    kl_loss = tf.reduce_mean(
        tf.reduce_sum(
            -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
            axis = 1,
        )
    )
    total_loss = reconstruction_loss + kl_loss

    return{
        "loss" : total_loss,
        "reconstruction_loss" : reconstruction_loss,
        "kl_loss" : kl_loss,
    }

In [ ]:
#Create a variational autoencoder
vae = VAE(encoder, decoder)

#Train the variational autoencoder

In [ ]:
#Compile the variational autoencoder

optimizers = optimizers.Adam(learning_rate = 0.0005)
vae.compile(optimizer = optimizer)

In [ ]:
#Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath = "./checkpoint",
    save_weights_only = False,
    save_freq = "epoch",
    monitor = "loss",
    mode = "min",
    save_best_only = True,
    verbose = 0,

)
tensorboard_callback = callbacks.TensorBoard(log_dir = "./logs")


In [ ]:
vae.fit(
    x_train,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE ,
    shuffle = True,
    validation_data = (x_test , x_test),
    callbacks = [model_checkpoint_callback, tensorboard_callback],
)

In [ ]:
#save the final models
vae.save("./models/vae")
encoder.save("./models/encoder")
decoder.save("./models/decoder")

#3. Reconstruct using the variational autoencoder

In [ ]:
#Select a subset of the test set
n_to_predict = 5000
example_images = x_test[:n_to_predict]
example_labels = y_test[:n_to_predict]


In [ ]:
#Create autoencoder predictions and dispaly
z_mean, z_log_var, reconstructions = vae.predict(example_images)
print("Example real clothing items")
